In [11]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
from qiskit.quantum_info import Pauli
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from qiskit import Aer
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy
import json
import time

In [12]:
def Energy2(params):
    circ=var_form_base.construct_circuit(parameters=params) 
    state=execute(circ,backend,shots=1).result().get_statevector() 
    E=(numpy.conjugate(state)@Hmat@state).real
    return E
def Energy1(params):
    circ=ansatz_circuit(params) #var_form_base.construct_circuit(parameters=params) 
    state=execute(circ,backend,shots=1).result().get_statevector() 
    E=(numpy.conjugate(state)@Hmat@state).real
    return E
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat 
def qubit_UCCSD_pool(*args,**kwargs):
    circ=QuantumCircuit(8)
    circ.x(0)
    circ.x(1)
    circ.x(4)
    circ.x(5)
    var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
    var_form_base.manage_hopping_operators()
    fer_pool_UCCSD=var_form_base.excitation_pool
    qubit_pool_UCCSD=[]
    for i in range(len(fer_pool_UCCSD)):
        for j in range(len(fer_pool_UCCSD[i].paulis)):
            qubit_pool_UCCSD.append(WeightedPauliOperator([[1j,fer_pool_UCCSD[i].paulis[j][1]]]))
    return numpy.array(qubit_pool_UCCSD)
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('jordan_wigner')
    return qubit_op
#Ansatz Circuit Construction
#construct initial state
def HF_init_state(U,return_MO_rotnOp=False):
    backend=Aer.get_backend('statevector_simulator')
    N=4
    # Qiskit implementation of Givens rotation
    def prepare_givens_rotated_state(givens):
        circ = QuantumCircuit(2*N)
        # Fill first N_f orbitals for each spin
        for i in range(N_f):
            circ.x(i)
            circ.x(i+N)
        for rots in givens:
            for tup in rots:
                #for spin down
                spin=0
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
                #for spin up
                spin=1
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
        final_state_vector=execute(circ,backend,shots=1024).result().get_statevector()      
        return circ,final_state_vector
    def cost_fn(angles):
        givens=[((1, 2, angles[0], 0.0),), ((0, 1, angles[1], 0.0), (2, 3, angles[2], 0.0)), ((1, 2, angles[3], 0.0),)]  
        #c=Givens_rot_circuit(givens)
        #state = QuantumState(N*2)
        #c.update_quantum_state(state)
        #state_vector=numpy.array(state.get_vector())
        c,state=prepare_givens_rotated_state(givens)
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(state),Hmat),state))
        return E
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot        
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix#qubitH.to_matrix(massive=True)
    E,V=numpy.linalg.eigh(eg_h1)
    energy = 2*sum(E[:int(N/2)])
    psi = V[:,:int(N/2)]
    Q = numpy.transpose(numpy.conjugate(psi))
    N_f=len(Q)
    givens_init = slater_determinant_preparation_circuit(Q)
    params_init=[]
    for k in range(len(givens_init)):
        for l in range(len(givens_init[k])):
            params_init.append(givens_init[k][l][2])        
    res = scipy.optimize.minimize(cost_fn, params_init, bounds=[[-numpy.pi,numpy.pi]]*4,method='L-BFGS-B')
    print("Final Hartree Fock Energy",res['fun'])
    #final slater determinant state vector obtained using quantum variational Hartree Fock using Given Rotations
    params_fin=res['x']
    #Final Givens Rotation Circuit obtained after implementing L-BFGS-B algorithm
    givens_fin=[((1, 2, params_fin[0], 0.0),), ((0, 1, params_fin[1], 0.0), (2, 3, params_fin[2], 0.0)), ((1, 2, params_fin[3], 0.0),)]
    HFcirc,HFstateVec=prepare_givens_rotated_state(givens_fin)
    if return_MO_rotnOp==True:
        givensOp=prepare_given_operator(givens_fin)
        return HFcirc,HFstateVec,givensOp
    return HFcirc,HFstateVec
def ref_state(*args,**kwargs):
    init_circ=QuantumCircuit(num_qubits)
    for i in range(num_sites//2):
        init_circ.x(i)
        init_circ.x(i+num_sites)
    return init_circ    
def PauliStringToMatrix(Label):
    Label=numpy.array(list(Label))
    LabelInds=numpy.where(Label!='I')[0]
    substring=''.join(list(Label[LabelInds]))
    UnitaryMat=Pauli.from_label(substring).to_matrix()
    return (UnitaryMat,LabelInds)
def UnitaryMatrixForm(params,ind):
    Label=list(PaulisAndMats.keys())[ind]
    UnitaryMat,qubits_to_act_on=PaulisAndMats[Label]
    UnitaryMatrix=numpy.cos(params[ind])*numpy.eye(2**len(qubits_to_act_on))-1j*numpy.sin(params[ind])*UnitaryMat
    return [UnitaryMatrix,qubits_to_act_on,Label]
def ansatz_circuit(params):
    circ=HFcirc.copy() #ref_state()
    UnitaryMatArr=Parallel(n_jobs=7,verbose=0)(delayed(UnitaryMatrixForm)(params,i) for i in range(len(params)))
    for i in range(len(UnitaryMatArr)):
        circ.unitary(UnitaryMatArr[i][0],list(UnitaryMatArr[i][1]),label=UnitaryMatArr[i][2])
    return circ    
U=7
num_qubits=8
num_sites=num_qubits//2
#construct Hamiltonian
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g
U=7
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)
HFcirc,HFstate=HF_init_state(U)
# #Hamiltonian for eg Model in MO basis
# #Setting up MO Hamiltonian
# Hlist=json.load(open('../7/FinalDataFiles/adaptVQE-opt-SMO-MO-Hamiltonian/HinMOrep1.json','r'))['h']
# penaltyList=json.load(open('../7/FinalDataFiles/adaptVQE-opt-SMO-MO-Hamiltonian/HinMOrep1.json','r'))['ntot']
# nume=json.load(open('../7/FinalDataFiles/adaptVQE-opt-SMO-MO-Hamiltonian/HinMOrep1.json','r'))['nume']
# Hlist=[Hlist[i].split('*') for i in range(len(Hlist))]
# qubitH=WeightedPauliOperator([[float(Hlist[i][0]),Pauli.from_label(Hlist[i][1])] for i in range(len(Hlist))])
# op2list=[penaltyList[i].split('*') for i in range(len(penaltyList))]
# op2=WeightedPauliOperator([[float(op2list[i][0]),Pauli.from_label(op2list[i][1])] for i in range(len(op2list))])
# Num=op_converter.to_matrix_operator(op2).dense_matrix
# with open('../chem_pot_for_Half_Fill.txt','r') as f:
#         lines=f.readlines()[1:]
#         for line in lines:
#             elems=line.split()
#             if int(elems[0])==U:
#                 muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
# chem_pot=numpy.zeros((2*num_sites,2*num_sites))
# for i in range(len(chem_pot)):
#     chem_pot[i][i]=-muHalf
# qubit_chem_pot=qubitOp(chem_pot,numpy.zeros((8,8,8,8)))
# qubitH=qubitH.add(qubit_chem_pot)
# Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
# w,v=numpy.linalg.eigh(Hmat)
# Eg=w[0]
# print("exact GS E",Eg)

Final Hartree Fock Energy -18.788335371650774


In [3]:
#qubit UCCSD pool
#qubit_pool_UCCSD=qubit_UCCSD_pool()
#constructing matrices for gradient computation
#MatrixRepOfPoolOps=list(map(commutatorsForGradient,qubit_pool_UCCSD))
#print("pool size",len(MatrixRepOfPoolOps))

In [17]:
label='IIIZIXZY'
init_circ=HFcirc.copy()
backend=Aer.get_backend('statevector_simulator')
var_form_base=UCCSD(8,num_particles=4, initial_state=init_circ,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base.manage_hopping_operators()
var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(label)]]))
Energy2([0.1]),Energy2([-0.1])

(-18.859868062448065, -18.690048685947104)

In [16]:
PaulisAndMats={label:[]}
PaulisAndMats[label]=PauliStringToMatrix(label)
Energy1([0.1]),Energy1([-0.1])

(-18.775788604163676, -18.775788604163676)

In [4]:
label=[qubit_pool_UCCSD[17].paulis[0][1].to_label(),qubit_pool_UCCSD[18].paulis[0][1].to_label()]
label

['IYZYIXZY', 'IXZXIXZY']

In [5]:
init_circ=ref_state()
backend=Aer.get_backend('statevector_simulator')
var_form_base=UCCSD(8,num_particles=4, initial_state=init_circ,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base.manage_hopping_operators()
var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli.from_label(label[0])]]))
var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli.from_label(label[1])]]))
Energy2([0.1,0.1]),Energy2([0.1,-0.1])

(-18.81009448440258, -18.788345427719992)

In [6]:
PaulisAndMats={label[0]:[],label[1]:[]}
PaulisAndMats[label[0]]=PauliStringToMatrix(label[0])
PaulisAndMats[label[1]]=PauliStringToMatrix(label[1])

In [7]:
Energy1([0.1,0.1]),Energy1([0.1,-0.1])

(-18.810090580802097, -18.78834542772)

In [81]:
print(numpy.where(PaulisAndMats[label]!=0))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]), array([45, 44, 47, 46, 41, 40, 43, 42, 37, 36, 39, 38, 33, 32, 35, 34, 61,
       60, 63, 62, 57, 56, 59, 58, 53, 52, 55, 54, 49, 48, 51, 50, 13, 12,
       15, 14,  9,  8, 11, 10,  5,  4,  7,  6,  1,  0,  3,  2, 29, 28, 31,
       30, 25, 24, 27, 26, 21, 20, 23, 22, 17, 16, 19, 18]))


In [1]:
def compute_gradient(params,Op):
    PaulisAndMats.append([Op.paulis[0][1].to_label(),PauliStringToMatrix(Op.paulis[0][1].to_label())])
    E1=Energy(params+[0.01])
    E2=Energy(params+[-0.01])
    grad=numpy.abs(E1-E2)/0.02
    PaulisAndMats.pop(-1)
    return grad